In [2]:
import numpy as np
import pandas as pd
import pandas._testing as tm

# GroupBy sorting


## data1

In [17]:
df2 = pd.DataFrame({"X": ["B", "B", "A", "A"], "Y": [1, 2, 3, 4]})
df2

,X,Y
0,B,1
1,B,2
2,A,3
3,A,4


In [18]:
df2.groupby(["X"]).sum()

,Y
X,
A,7
B,3


In [16]:
df2.groupby(["X"], sort=False).sum()

,Y
X,
B,3
A,7


## data2

In [24]:
df3 = pd.DataFrame({"X": ["A", "B", "A", "B"], "Y": [1, 4, 3, 2]})
df3

,X,Y
0,A,1
1,B,4
2,A,3
3,B,2


In [25]:
df3.groupby(["X"]).get_group("A")

,X,Y
0,A,1
2,A,3


In [26]:
df3.groupby(["X"]).get_group("B")

,X,Y
1,B,4
3,B,2


## dropna

In [27]:
df_list = [[1, 2, 3], [1, None, 4], [2, 1, 3], [1, 2, 2]]
df_dropna = pd.DataFrame(df_list, columns=["a", "b", "c"])
df_dropna

,a,b,c
0,1,2.0,3
1,1,NaN,4
2,2,1.0,3
3,1,2.0,2


In [28]:
df_dropna.groupby(by=["b"], dropna=True).sum()

,a,c
b,,
1.0,2,3
2.0,2,5


In [30]:
df_dropna.groupby(by=["b"], dropna=False).sum()

,a,c
b,,
1.0,2,3
2.0,2,5
NaN,1,4


### GroupBy object attributes


In [31]:
df.groupby("A").groups

{'bar': [1, 3, 5], 'foo': [0, 2, 4, 6, 7]}

In [33]:
def get_letter_type(letter):
    if letter.lower() in 'aeiou':
        return 'vowel'
    else:
        return 'consonant'

In [34]:
df.groupby(get_letter_type, axis=1).groups

{'consonant': ['B', 'C', 'D'], 'vowel': ['A']}

In [37]:
grouped = df.groupby(["A", "B"])
grouped.groups

{('bar', 'one'): [1], ('bar', 'three'): [3], ('bar', 'two'): [5], ('foo', 'one'): [0, 6], ('foo', 'three'): [7], ('foo', 'two'): [2, 4]}

In [38]:
len(grouped)

6

### GroupBy with MultiIndex

In [39]:
arrays = [
    ["bar", "bar", "baz", "baz", "foo", "foo", "qux", "qux"],
    ["one", "two", "one", "two", "one", "two", "one", "two"],
]
index = pd.MultiIndex.from_arrays(arrays, names=["first", "second"])
s = pd.Series(np.random.randn(8), index=index)
s

first  second
bar    one      -1.745861
       two       1.546639
baz    one      -0.207495
       two       1.319016
foo    one      -0.811047
       two       0.908056
qux    one       0.151323
       two      -2.634327
dtype: float64

In [45]:
grouped = s.groupby(level=0)
grouped.sum()

first
bar   -0.199223
baz    1.111521
foo    0.097009
qux   -2.483003
dtype: float64

In [46]:
grouped = s.groupby(level=1)
grouped.sum()

second
one   -2.613080
two    1.139383
dtype: float64

In [48]:
s.groupby(level="second").sum()

second
one   -2.613080
two    1.139383
dtype: float64

#### Multiple levels

In [50]:
s.groupby(level=["first", "second"]).sum()

first  second
bar    one      -1.745861
       two       1.546639
baz    one      -0.207495
       two       1.319016
foo    one      -0.811047
       two       0.908056
qux    one       0.151323
       two      -2.634327
dtype: float64

In [52]:
s.groupby(["first", "second"]).sum()

first  second
bar    one      -1.745861
       two       1.546639
baz    one      -0.207495
       two       1.319016
foo    one      -0.811047
       two       0.908056
qux    one       0.151323
       two      -2.634327
dtype: float64

### Grouping DataFrame with Index levels and columns

In [54]:
arrays = [
    ["bar", "bar", "baz", "baz", "foo", "foo", "qux", "qux"],
    ["one", "two", "one", "two", "one", "two", "one", "two"],
]
index = pd.MultiIndex.from_arrays(arrays, names=["first", "second"])
df = pd.DataFrame({"A": [1, 1, 1, 1, 2, 2, 3, 3], "B": np.arange(8)}, index=index)
df

A  B
first second      
bar   one     1  0
      two     1  1
baz   one     1  2
      two     1  3
foo   one     2  4
      two     2  5
qux   one     3  6
      two     3  7

In [55]:
df.groupby([pd.Grouper(level=1), "A"]).sum()

B
second A   
one    1  2
       2  4
       3  6
two    1  4
       2  5
       3  7

In [57]:
df.groupby([pd.Grouper(level="second"), "A"]).sum()

B
second A   
one    1  2
       2  4
       3  6
two    1  4
       2  5
       3  7

In [62]:
df.groupby(["second", "A"]).sum()

B
second A   
one    1  2
       2  4
       3  6
two    1  4
       2  5
       3  7

### DataFrame column selection in GroupBy

In [170]:
df = pd.DataFrame(
    {
        "A": ["foo", "bar", "foo", "bar", "foo", "bar", "foo", "foo"],
        "B": ["one", "one", "two", "three", "two", "two", "one", "three"],
        "C": np.random.randn(8),
        "D": np.random.randn(8),
    }
)
df["C"].groupby(df["A"])

# Iterating through groups


In [171]:
df

,A,B,C,D
0,foo,one,0.682370,-0.822012
1,bar,one,-0.115472,0.144928
2,foo,two,-0.967031,0.210112
3,bar,three,-0.493386,1.457054
4,foo,two,-0.116610,0.795651
5,bar,two,0.822412,0.668128
6,foo,one,0.204052,0.534064
7,foo,three,-0.146871,1.025275


In [172]:
grouped = df.groupby('A')
for name, group in grouped:
    print(name)
    print(group)

bar
     A      B         C         D
1  bar    one -0.115472  0.144928
3  bar  three -0.493386  1.457054
5  bar    two  0.822412  0.668128
foo
     A      B         C         D
0  foo    one  0.682370 -0.822012
2  foo    two -0.967031  0.210112
4  foo    two -0.116610  0.795651
6  foo    one  0.204052  0.534064
7  foo  three -0.146871  1.025275


In [173]:
for name, group in df.groupby(['A', 'B']):
    print(name)
    print(group)

('bar', 'one')
     A    B         C         D
1  bar  one -0.115472  0.144928
('bar', 'three')
     A      B         C         D
3  bar  three -0.493386  1.457054
('bar', 'two')
     A    B         C         D
5  bar  two  0.822412  0.668128
('foo', 'one')
     A    B         C         D
0  foo  one  0.682370 -0.822012
6  foo  one  0.204052  0.534064
('foo', 'three')
     A      B         C         D
7  foo  three -0.146871  1.025275
('foo', 'two')
     A    B         C         D
2  foo  two -0.967031  0.210112
4  foo  two -0.116610  0.795651


### Selecting a group


In [174]:
grouped = df.groupby('A')
grouped.get_group("bar")

,A,B,C,D
1,bar,one,-0.115472,0.144928
3,bar,three,-0.493386,1.457054
5,bar,two,0.822412,0.668128


In [175]:
df

,A,B,C,D
0,foo,one,0.682370,-0.822012
1,bar,one,-0.115472,0.144928
2,foo,two,-0.967031,0.210112
3,bar,three,-0.493386,1.457054
4,foo,two,-0.116610,0.795651
5,bar,two,0.822412,0.668128
6,foo,one,0.204052,0.534064
7,foo,three,-0.146871,1.025275


In [176]:
df.groupby(["A", "B"]).get_group(("bar", "one"))

,A,B,C,D
1,bar,one,-0.115472,0.144928


In [177]:
df.groupby("A").get_group("foo")

,A,B,C,D
0,foo,one,0.682370,-0.822012
2,foo,two,-0.967031,0.210112
4,foo,two,-0.116610,0.795651
6,foo,one,0.204052,0.534064
7,foo,three,-0.146871,1.025275


### Aggregation

In [178]:
df

,A,B,C,D
0,foo,one,0.682370,-0.822012
1,bar,one,-0.115472,0.144928
2,foo,two,-0.967031,0.210112
3,bar,three,-0.493386,1.457054
4,foo,two,-0.116610,0.795651
5,bar,two,0.822412,0.668128
6,foo,one,0.204052,0.534064
7,foo,three,-0.146871,1.025275


In [179]:
grouped = df.groupby("A")
grouped.aggregate(np.sum)

,C,D
A,,
bar,0.213554,2.270110
foo,-0.344090,1.743088


In [180]:
grouped = df.groupby(["A", "B"])
grouped.aggregate(np.sum)

C         D
A   B                        
bar one   -0.115472  0.144928
    three -0.493386  1.457054
    two    0.822412  0.668128
foo one    0.886422 -0.287948
    three -0.146871  1.025275
    two   -1.083641  1.005762

In [181]:
# df.groupby(['A', 'B']).get_group(('bar', 'one')).sum()

In [182]:
grouped = df.groupby(["A", "B"], as_index=False)
grouped.aggregate(np.sum)

,A,B,C,D
0,bar,one,-0.115472,0.144928
1,bar,three,-0.493386,1.457054
2,bar,two,0.822412,0.668128
3,foo,one,0.886422,-0.287948
4,foo,three,-0.146871,1.025275
5,foo,two,-1.083641,1.005762


In [183]:
df.groupby("A", as_index=False).sum()

,A,C,D
0,bar,0.213554,2.270110
1,foo,-0.344090,1.743088


In [184]:
df.groupby(["A", "B"]).sum()

C         D
A   B                        
bar one   -0.115472  0.144928
    three -0.493386  1.457054
    two    0.822412  0.668128
foo one    0.886422 -0.287948
    three -0.146871  1.025275
    two   -1.083641  1.005762

In [185]:
df.groupby(["A", "B"]).sum().reset_index()

,A,B,C,D
0,bar,one,-0.115472,0.144928
1,bar,three,-0.493386,1.457054
2,bar,two,0.822412,0.668128
3,foo,one,0.886422,-0.287948
4,foo,three,-0.146871,1.025275
5,foo,two,-1.083641,1.005762


In [200]:
df.groupby(["A", "B"]).sum()

C         D
A   B                        
bar one   -0.115472  0.144928
    three -0.493386  1.457054
    two    0.822412  0.668128
foo one    0.886422 -0.287948
    three -0.146871  1.025275
    two   -1.083641  1.005762

In [186]:
grouped.size()

,A,B,size
0,bar,one,1
1,bar,three,1
2,bar,two,1
3,foo,one,2
4,foo,three,1
5,foo,two,2


In [187]:
grouped.describe()

C                                                                        \
  count      mean       std       min       25%       50%       75%       max   
0   1.0 -0.115472       NaN -0.115472 -0.115472 -0.115472 -0.115472 -0.115472   
1   1.0 -0.493386       NaN -0.493386 -0.493386 -0.493386 -0.493386 -0.493386   
2   1.0  0.822412       NaN  0.822412  0.822412  0.822412  0.822412  0.822412   
3   2.0  0.443211  0.338222  0.204052  0.323631  0.443211  0.562790  0.682370   
4   1.0 -0.146871       NaN -0.146871 -0.146871 -0.146871 -0.146871 -0.146871   
5   2.0 -0.541820  0.601339 -0.967031 -0.754426 -0.541820 -0.329215 -0.116610   

      D                                                                        
  count      mean       std       min       25%       50%       75%       max  
0   1.0  0.144928       NaN  0.144928  0.144928  0.144928  0.144928  0.144928  
1   1.0  1.457054       NaN  1.457054  1.457054  1.457054  1.457054  1.457054  
2   1.0  0.668128       NaN  0.668128  0.668128  0.668128  0.668128  0.668128  
3   2.0 -0.143974  0.958891 -0.822012 -0.482993 -0.143974  0.195045  0.534064  
4   1.0  1.025275       NaN  1.025275  1.025275  1.025275  1.025275  1.025275  
5   2.0  0.502881  0.414039  0.210112  0.356496  0.502881  0.649266  0.795651

In [188]:
ll = [['foo', 1], ['foo', 2], ['foo', 2], ['bar', 1], ['bar', 1]]
df4 = pd.DataFrame(ll, columns=["A", "B"])
df4

,A,B
0,foo,1
1,foo,2
2,foo,2
3,bar,1
4,bar,1


In [189]:
df4.groupby("A").nunique()

,B
A,
bar,1
foo,2


In [190]:
df4.groupby("A")["B"].nunique()

A
bar    1
foo    2
Name: B, dtype: int64

# Applying multiple functions at once


In [191]:
df

,A,B,C,D
0,foo,one,0.682370,-0.822012
1,bar,one,-0.115472,0.144928
2,foo,two,-0.967031,0.210112
3,bar,three,-0.493386,1.457054
4,foo,two,-0.116610,0.795651
5,bar,two,0.822412,0.668128
6,foo,one,0.204052,0.534064
7,foo,three,-0.146871,1.025275


In [192]:
grouped = df.groupby("A")
grouped["C"].agg([np.sum, np.mean, np.std])

,sum,mean,std
A,,,
bar,0.213554,0.071185,0.677467
foo,-0.344090,-0.068818,0.602890


In [193]:
grouped[["C", "D"]].agg([np.sum, np.mean, np.std])

C                             D                    
          sum      mean       std       sum      mean       std
A                                                              
bar  0.213554  0.071185  0.677467  2.270110  0.756703  0.660532
foo -0.344090 -0.068818  0.602890  1.743088  0.348618  0.721396

In [194]:
(
    grouped["C"]
    .agg([np.sum, np.mean, np.std])
    .rename(columns={"sum": "foo", "mean": "bar", "std": "baz"})
)

,foo,bar,baz
A,,,
bar,0.213554,0.071185,0.677467
foo,-0.344090,-0.068818,0.602890


In [195]:
df

,A,B,C,D
0,foo,one,0.682370,-0.822012
1,bar,one,-0.115472,0.144928
2,foo,two,-0.967031,0.210112
3,bar,three,-0.493386,1.457054
4,foo,two,-0.116610,0.795651
5,bar,two,0.822412,0.668128
6,foo,one,0.204052,0.534064
7,foo,three,-0.146871,1.025275


In [136]:
animals = pd.DataFrame(
    {
        "kind": ["cat", "dog", "cat", "dog"],
        "height": [9.1, 6.0, 9.5, 34.0],
        "weight": [7.9, 7.5, 9.9, 198.0],
    }
)

In [146]:
df = pd.DataFrame({"a": [1, 0, 0], "b": [0, 1, 0], "c": [1, 0, 0], "d": [2, 3, 4]})
df

,a,b,c,d
0,1,0,1,2
1,0,1,0,3
2,0,0,0,4


In [147]:
df.sum()

a    1
b    1
c    1
d    9
dtype: int64

In [150]:
df.sum()

a    1
b    1
c    1
d    9
dtype: int64

In [159]:
dff = pd.DataFrame({"A": np.arange(8), "B": list("aabbbbcc")})
dff

,A,B
0,0,a
1,1,a
2,2,b
3,3,b
4,4,b
5,5,b
6,6,c
7,7,c


In [166]:
dff.groupby('A').sum()

,B
A,
0,a
1,a
2,b
3,b
4,b
5,b
6,c
7,c


In [167]:
dff.groupby('A').count()

,B
A,
0,1
1,1
2,1
3,1
4,1
5,1
6,1
7,1


In [169]:
dff.groupby('A').filter(lambda x: len(x) >= 2)

,A,B
